<a href="https://colab.research.google.com/github/Shun0212/CodeBERTPretrained/blob/main/codemorphv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from transformers import AutoModelForMaskedLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

model_name = "Shuu12121/CodeMorph-BERTv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

!pip install datasets

# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
import torch
import random
from datasets import load_dataset, Dataset
from tokenizers import BertWordPieceTokenizer
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
os.chdir('/content/drive/MyDrive/metamorphosis')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/728k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/254M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [ ]:
import torch
import numpy as np
from transformers import AutoModelForMaskedLM, AutoTokenizer, RobertaForMaskedLM, BertForMaskedLM
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import random


import re
from datasets import load_dataset

def remove_multiline_strings(code):
    """
    Pythonコードからマルチライン文字列（""" """ や ''' '''）を削除する関数
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def func_load_codes_from_xglue(num_samples: int = 10000):
    """
    google/code_x_glue_tc_nl_code_search_adv の Python コードデータ（code）を抽出し、
    マルチライン文字列（""" """ や ''' '''）を削除してリストに格納します。

    Args:
        num_samples (int): 抽出するサンプル数。0 以下（例: -1）を指定すると全サンプルを使用します。

    Returns:
        List[str]: 抽出したコードのリスト
    """
    dataset = load_dataset("google/code_x_glue_tc_nl_code_search_adv", split="train")
    code_samples = []
    for item in dataset:
        code = item.get("code", "")
        # マルチライン文字列を削除
        cleaned_code = remove_multiline_strings(code)
        # 空文字でなければ追加
        if cleaned_code and len(cleaned_code.strip()) > 0:
            code_samples.append(cleaned_code)
        # num_samples が正の値の場合のみ打ち切り
        if num_samples > 0 and len(code_samples) >= num_samples:
            break
    print(f"抽出したコードサンプル数: {len(code_samples)}")
    return code_samples


def func_tokenize_dataset(dataset, tokenizer, max_length: int = 512):

    def tokenize_function(example):
        texts = example["code"]
        # 各サンプルが None または空文字なら "" に置き換える
        texts = [text if text is not None and text.strip() != "" else "" for text in texts]
        # batched なのでリストをまとめて処理（返り値は dict になります）
        return tokenizer(texts, truncation=True, max_length=max_length)

    # func_code_string 以外のカラムは削除
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=[col for col in dataset.column_names if col != "code"]
    )
    return tokenized_dataset



# --- 訓練用データセットの準備 ---

# --- 評価用データセットの準備 ---
# 例として validation split を使用（存在しない場合は test など適宜変更してください）
raw_eval_dataset = load_dataset("google/code_x_glue_tc_nl_code_search_adv", split="test")
tokenized_eval_dataset = func_tokenize_dataset(raw_eval_dataset, tokenizer, max_length=512)




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")



def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="func_documentation_string", code_field="func_code_string"):
    num_examples = min(len(dataset), max_examples)
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(len(dataset))]
    queries = []
    correct_code_indices = []

    for i in range(num_examples):
        ex = dataset[i]
        query_text = ex[query_field]
        queries.append(query_text)
        correct_code_indices.append(i)

    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes[:num_examples]:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        candidate_pool_embeddings = []
        candidate_pool_embeddings.append(all_code_embeddings[correct_code_indices[i]])
        incorrect_code_indices = []
        while len(incorrect_code_indices) < pool_size - 1:
            rand_index = random.randint(0, num_examples - 1)
            if rand_index != correct_code_indices[i] and rand_index not in incorrect_code_indices:
                incorrect_code_indices.append(rand_index)
        for incorrect_index in incorrect_code_indices:
            candidate_pool_embeddings.append(all_code_embeddings[incorrect_index])
        candidate_pool_embeddings = np.stack(candidate_pool_embeddings, axis=0)
        sims = cosine_similarity(query_embedding, candidate_pool_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")
    metrics = calculate_metrics(sim_matrix)
    return metrics


def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    if isinstance(model, BertForMaskedLM):
        outputs = model.bert(**inputs)
    elif isinstance(model, RobertaForMaskedLM):
        outputs = model.roberta(**inputs)
    else:
        raise ValueError("Model must be BertForMaskedLM or RobertaForMaskedLM.")
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.detach().cpu().numpy()

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "map": 0.0,
        "f1@k": {k: 0.0 for k in k_values},
        "r_precision": 0.0,
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }
    for i in range(num_queries):
        sims = sim_matrix[i]
        ranked_indices = np.argsort(-sims)
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        is_correct_in_top_k = {k: correct_rank <= k for k in k_values}
        metrics["mrr"] += 1.0 / correct_rank
        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1
            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0
            metrics["success_rate@k"][k] += 1.0 if is_correct_in_top_k[k] else 0.0
            metrics["query_coverage@k"][k] += 1.0 if is_correct_in_top_k[k] else 0.0
        metrics["map"] += calculate_average_precision(ranked_indices)
        metrics["r_precision"] += calculate_r_precision(ranked_indices)
    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["recall@k"][k] /= num_queries
        metrics["precision@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def calculate_average_precision(ranked_indices):
    correct_rank = np.where(ranked_indices == 0)[0][0] + 1
    ap = 0.0
    for k in range(1, len(ranked_indices) + 1):
        if k == correct_rank:
            ap += 1.0 / k
    return ap

def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_r_precision(ranked_indices):
    r = 1
    correct_in_top_r = 0
    for i in range(r):
        if ranked_indices[i] == 0:
            correct_in_top_r += 1
    return correct_in_top_r / r

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg



if __name__ == "__main__":
    print("\ngoogle/code_x_glue_ct_code_to_text データセット (Test) をロードします...")
    tc_dataset = load_dataset("google/code_x_glue_ct_code_to_text","python", split="test" , trust_remote_code=True)
    subset_tc = tc_dataset.select(range(14918))
    model_name1 = "Shuu12121/CodeMorph-BERT"
    print(f"\n{model_name1} を google/code_x_glue_ct_code_to_text で評価します (候補プールサイズ: 100)...")
    tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
    model1 = AutoModelForMaskedLM.from_pretrained(model_name1)
    model1.to(device)
    metrics_tc1 = evaluate_code_search(model1, tokenizer1, subset_tc, device, max_examples=14918, pool_size=100, query_field="docstring", code_field="code")
    print(f"{model_name1} の評価指標 (google/):")
    for name, value in metrics_tc1.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")
    model_name2 = "microsoft/codebert-base-mlm"
    print(f"\n{model_name2} を google/ で評価します (候補プールサイズ: 100)...")
    tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
    model2 = AutoModelForMaskedLM.from_pretrained(model_name2)
    model2.to(device)
    metrics_tc2 = evaluate_code_search(model2, tokenizer2, subset_tc, device, max_examples=14918, pool_size=100, query_field="docstring", code_field="code")
    print(f"{model_name2} の評価指標 (google/):")
    for name, value in metrics_tc2.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")


    model_name3 = "Shuu12121/CodeMorph-BERTv2"
    print(f"\n{model_name3} を google/ で評価します (候補プールサイズ: 100)...")
    tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
    model3 = AutoModelForMaskedLM.from_pretrained(model_name3)
    model3.to(device)
    metrics_tc1 = evaluate_code_search(model3, tokenizer3, subset_tc, device, max_examples=14918, pool_size=100, query_field="docstring", code_field="code")
    print(f"{model_name3} の評価指標 (google/):")
    for name, value in metrics_tc1.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")

Map:   0%|          | 0/19210 [00:00<?, ? examples/s]

使用デバイス: cuda

google/code_x_glue_ct_code_to_text データセット (Test) をロードします...

Shuu12121/CodeMorph-BERT を google/code_x_glue_ct_code_to_text で評価します (候補プールサイズ: 100)...


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/728k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.
microsoft/codebert-base-mlm の評価指標 (google/):
  mrr: 0.5231
  recall@k@1: 0.4350
  recall@k@5: 0.6028
  recall@k@10: 0.6907
  recall@k@50: 0.9445
  recall@k@100: 1.0000
  precision@k@1: 0.4350
  precision@k@5: 0.4981
  precision@k@10: 0.5097
  precision@k@50: 0.5222
  precision@k@100: 0.5231
  ndcg@k@1: 0.4350
  ndcg@k@5: 0.5243
  ndcg@k@10: 0.5525
  ndcg@k@50: 0.6094
  ndcg@k@100: 0.6185
  map: 0.5231
  f1@k@1: 0.4350
  f1@k@5: 0.5249
  f1@k@10: 0.5453
  f1@k@50: 0.5689
  f1@k@100: 0.5706
  r_precision: 0.4350
  success_rate@k@1: 0.4350
  success_rate@k@5: 0.6028
  success_rate@k@10: 0.6907
  success_rate@k@50: 0.9445
  success_rate@k@100: 1.0000
  query_coverage@k@1: 0.4350
  query_coverage@k@5: 0.6028
  query_coverage@k@10: 0.6907
  query_coverage@k@50: 0.9445
  query_coverage@k@100: 1.0000

Shuu12121/CodeMorph-BERTv2 を google/ で評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度

In [ ]:
import torch
import numpy as np
from transformers import AutoModelForMaskedLM, AutoTokenizer, RobertaForMaskedLM, BertForMaskedLM
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import random
import re

def remove_multiline_strings(code):
    """
    Pythonコードからマルチライン文字列（ダブルクォート3つまたはシングルクォート3つで囲まれた部分）を削除する関数
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def func_load_codes_from_xglue(num_samples: int = 10000):
    """
    google/code_x_glue_tc_nl_code_search_adv の Python コードデータ（code）を抽出し、
    マルチライン文字列（""" """ や ''' '''）を削除してリストに格納します。

    Args:
        num_samples (int): 抽出するサンプル数。0 以下（例: -1）を指定すると全サンプルを使用します。

    Returns:
        List[str]: 抽出したコードのリスト
    """
    dataset = load_dataset("google/code_x_glue_tc_nl_code_search_adv", split="train")
    code_samples = []
    for item in dataset:
        code = item.get("code", "")
        # マルチライン文字列を削除
        cleaned_code = remove_multiline_strings(code)
        # 空文字でなければ追加
        if cleaned_code and len(cleaned_code.strip()) > 0:
            code_samples.append(cleaned_code)
        # num_samples が正の値の場合のみ打ち切り
        if num_samples > 0 and len(code_samples) >= num_samples:
            break
    print(f"抽出したコードサンプル数: {len(code_samples)}")
    return code_samples


def func_tokenize_dataset(dataset, tokenizer, max_length: int = 512):

    def tokenize_function(example):
        texts = example["code"]
        # 各サンプルが None または空文字なら "" に置き換える
        texts = [text if text is not None and text.strip() != "" else "" for text in texts]
        # batched なのでリストをまとめて処理（返り値は dict になります）
        return tokenizer(texts, truncation=True, max_length=max_length)

    # func_code_string 以外のカラムは削除
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=[col for col in dataset.column_names if col != "code"]
    )
    return tokenized_dataset

def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="func_documentation_string", code_field="func_code_string"):
    """
    指定したデータセットに対して、コード検索タスクの各種評価指標を計算する関数
    （※正解候補は候補プールの先頭に固定しています）
    """
    num_examples = min(len(dataset), max_examples)
    # 全サンプルのコードからマルチライン文字列を除去してリストに格納
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(len(dataset))]
    queries = []
    # 正解候補のインデックスは各サンプルの位置（0～num_examples-1）とする
    for i in range(num_examples):
        ex = dataset[i]
        query_text = ex[query_field]
        queries.append(query_text)
    # 各コードの埋め込みを計算
    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes[:num_examples]:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    # (num_examples, embedding_dim) の2次元配列に統合
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        # 候補プール作成：正解候補（インデックス i）を必ず先頭に配置し、残りをランダムサンプル
        candidate_indices = [i] + random.sample([j for j in range(num_examples) if j != i], pool_size - 1)
        candidate_pool_embeddings = all_code_embeddings[candidate_indices]
        sims = cosine_similarity(query_embedding, candidate_pool_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")
    # 指標計算（候補プール内の正解は必ずインデックス0と仮定）
    metrics = calculate_metrics(sim_matrix)
    return metrics

def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    """
    モデルのCLSトークン埋め込みを取得する関数
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    if isinstance(model, BertForMaskedLM):
        outputs = model.bert(**inputs)
    elif isinstance(model, RobertaForMaskedLM):
        outputs = model.roberta(**inputs)
    else:
        raise ValueError("Model must be BertForMaskedLM or RobertaForMaskedLM.")
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.detach().cpu().numpy()

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    """
    MRR、Recall@k、Precision@k、NDCG@k、MAP、F1@k、R-Precision、Success Rate@k、Query Coverage@k を計算する
    ※候補プール内の正解候補は常にインデックス0と仮定
    """
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "map": 0.0,
        "f1@k": {k: 0.0 for k in k_values},
        "r_precision": 0.0,
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }
    for i in range(num_queries):
        sims = sim_matrix[i]
        # 候補プール内の順位（降順ソート）
        ranked_indices = np.argsort(-sims)
        # 正解候補（インデックス0）のランク位置（1始まり）
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        metrics["mrr"] += 1.0 / correct_rank

        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1
            # NDCGの計算
            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0
            metrics["success_rate@k"][k] += 1.0 if correct_rank <= k else 0.0
            metrics["query_coverage@k"][k] += 1.0 if correct_rank <= k else 0.0

        # MAP（ここでは正解が1件なので1/correct_rank と同じ）
        metrics["map"] += 1.0 / correct_rank
        # R-Precision（R=1の場合は、正解がトップにあるかどうか）
        metrics["r_precision"] += 1.0 if correct_rank == 1 else 0.0

    # 各指標の平均化
    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["recall@k"][k] /= num_queries
        metrics["precision@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg

if __name__ == "__main__":
    # 評価データセットのロード（ここでは google/code_x_glue_ct_code_to_text を使用）
    print("\ngoogle/code_x_glue_ct_code_to_text データセット (Test) をロードします...")
    tc_dataset = load_dataset("google/code_x_glue_ct_code_to_text", "python", split="test", trust_remote_code=True)
    # 適当な件数をサブセットとして選択（例：最初の14918件）
    subset_tc = tc_dataset.select(range(14918))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用デバイス: {device}")

    # 評価対象モデル1: Shuu12121/CodeMorph-BERT
    model_name1 = "Shuu12121/CodeMorph-BERT"
    print(f"\n{model_name1} を google/code_x_glue_ct_code_to_text で評価します (候補プールサイズ: 100)...")
    tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
    model1 = AutoModelForMaskedLM.from_pretrained(model_name1)
    model1.to(device)
    # メインではフィールド名を "docstring" と "code" に上書き
    metrics_tc1 = evaluate_code_search(model1, tokenizer1, subset_tc, device, max_examples=14918, pool_size=100,
                                       query_field="docstring", code_field="code")
    print(f"{model_name1} の評価指標 (google/):")
    for name, value in metrics_tc1.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")

    # 評価対象モデル2: microsoft/codebert-base-mlm
    model_name2 = "microsoft/codebert-base-mlm"
    print(f"\n{model_name2} を google/ で評価します (候補プールサイズ: 100)...")
    tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
    model2 = AutoModelForMaskedLM.from_pretrained(model_name2)
    model2.to(device)
    metrics_tc2 = evaluate_code_search(model2, tokenizer2, subset_tc, device, max_examples=14918, pool_size=100,
                                       query_field="docstring", code_field="code")
    print(f"{model_name2} の評価指標 (google/):")
    for name, value in metrics_tc2.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")

    # 評価対象モデル3: Shuu12121/CodeMorph-BERTv2
    model_name3 = "Shuu12121/CodeMorph-BERTv2"
    print(f"\n{model_name3} を google/ で評価します (候補プールサイズ: 100)...")
    tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
    model3 = AutoModelForMaskedLM.from_pretrained(model_name3)
    model3.to(device)
    metrics_tc3 = evaluate_code_search(model3, tokenizer3, subset_tc, device, max_examples=14918, pool_size=100,
                                       query_field="docstring", code_field="code")
    print(f"{model_name3} の評価指標 (google/):")
    for name, value in metrics_tc3.items():
        if isinstance(value, dict):
            for k, v in value.items():
                print(f"  {name}@{k}: {v:.4f}")
        else:
            print(f"  {name}: {value:.4f}")



google/code_x_glue_ct_code_to_text データセット (Test) をロードします...
使用デバイス: cuda

Shuu12121/CodeMorph-BERT を google/code_x_glue_ct_code_to_text で評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.
Shuu12121/CodeMorph-BERT の評価指標 (google/):
  mrr: 0.6827
  recall@k@1: 0.5896
  recall@k@5: 0.7923
  recall@k@10: 0.8583
  recall@k@50: 0.9759
  recall@k@100: 1.0000
  precision@k@1: 0.5896
  precision@k@5: 0.6675
  precision@k@10: 0.6764
  precision@k@50: 0.6824
  precision@k@100: 0.6827
  ndcg@k@1: 0.5896
  ndcg@k@5: 0.6988
  ndcg@k@10: 0.7202
  ndcg@k@50: 0.7467
  ndcg@k@100: 0.7507
  map: 0.6827
  f1@k@1: 0.5896
  f1@k@5: 0.7000
  f1@k@10: 0.7156
  f1@k@50: 0.7268
  f1@k@100: 0.7275
  r_precision: 0.5896
  success_rate@k@1: 0.5896
  success_rate@k@5: 0.7923
  success_rate@k@10: 0.8583
  success_rate@k@50: 0.9759
  success_rate@k@100: 1.0000
  query_coverage@k@1: 0.5896
  query_coverage@k@5: 0.7923
  query_coverage@k@10: 0.8583
  query_coverage@k@50: 0.975

Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.
microsoft/codebert-base-mlm の評価指標 (google/):
  mrr: 0.5242
  recall@k@1: 0.4361
  recall@k@5: 0.6051
  recall@k@10: 0.6897
  recall@k@50: 0.9449
  recall@k@100: 1.0000
  precision@k@1: 0.4361
  precision@k@5: 0.4997
  precision@k@10: 0.5108
  precision@k@50: 0.5234
  precision@k@100: 0.5242
  ndcg@k@1: 0.4361
  ndcg@k@5: 0.5260
  ndcg@k@10: 0.5532
  ndcg@k@50: 0.6103
  ndcg@k@100: 0.6194
  map: 0.5242
  f1@k@1: 0.4361
  f1@k@5: 0.5266
  f1@k@10: 0.5463
  f1@k@50: 0.5700
  f1@k@100: 0.5716
  r_precision: 0.4361
  success_rate@k@1: 0.4361
  success_rate@k@5: 0.6051
  success_rate@k@10: 0.6897
  success_rate@k@50: 0.9449
  success_rate@k@100: 1.0000
  query_coverage@k@1: 0.4361
  query_coverage@k@5: 0.6051
  query_coverage@k@10: 0.6897
  query_coverage@k@50: 0.9449
  query_coverage@k@100: 1.0000

Shuu12121/CodeMorph-BERTv2 を google/ で評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度